In [24]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

In [12]:
##################################Set up files location and names
path='../Resources'

mp_df='monkeypox_df.csv'
mp_daily='Daily_Country_Wise_Confirmed_Cases.csv'
mp_worldwide='Monkey_Pox_Cases_Worldwide.csv'
mp_casedetection='Worldwide_Case_Detection_Timeline.csv'




In [13]:
os.chdir(path)

In [22]:
############################# mp_df data handling ################################################
individual_transformed=pd.read_csv(mp_df)
individual_transformed.rename(columns={'Hospitalised (Y/N/NA)':'isHospitalised','Isolated (Y/N/NA)':'isIsolated','Travel_history (Y/N/NA)':'hasTravelHistory'},inplace=True)
individual_transformed=individual_transformed.drop(individual_transformed.iloc[:, 20:30],axis = 1)

In [23]:
individual_transformed.head()

,ID,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,...,Date_hospitalisation,isIsolated,Date_isolation,Outcome,Contact_comment,Contact_ID,Contact_location,hasTravelHistory,Travel_history_entry,Country_ISO3
0,1.0,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-04-29,2022-05-06,rash,...,2022-05-04,Y,2022-05-04,NaN,NaN,NaN,NaN,Y,2022-05-04,GBR
1,2.0,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-05-05,2022-05-12,rash,...,2022-05-06,Y,2022-05-09,NaN,Index Case of household cluster,3.0,Household,N,NaN,GBR
2,3.0,confirmed,London,London,England,NaN,NaN,2022-04-30,2022-05-13,vesicular rash,...,NaN,Y,NaN,NaN,NaN,2.0,Household,N,NaN,GBR
3,4.0,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,Y,NaN,NaN,Under investigation,NaN,NaN,N,NaN,GBR
4,5.0,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,Y,NaN,NaN,Under investigation,NaN,NaN,N,NaN,GBR


In [15]:
############################# mp_daily data handling ################################################
daily=pd.read_csv(mp_daily)
daily_pivoted=pd.melt(daily, id_vars='Country')
daily_pivoted.rename(columns = {'variable':'Date_confirmation', 'Country':'Country', 'value':'Confirmed_Cases'}, inplace = True)
daily_transformed=daily_pivoted

In [19]:
daily_transformed.head()

,Country,Date_confirmation,Confirmed_Cases
0,England,2022-05-06,1
1,Portugal,2022-05-06,0
2,Spain,2022-05-06,0
3,United States,2022-05-06,0
4,Germany,2022-05-06,0


In [16]:
############################# mp_worldwide data handling ################################################
worldwide_transformed=pd.read_csv(mp_worldwide)

In [20]:
worldwide_transformed.head()

,Country,Confirmed_Cases,Suspected_Cases,Hospitalized,Travel_History_Yes,Travel_History_No
0,England,287.0,0.0,5.0,2.0,7.0
1,Portugal,143.0,0.0,0.0,0.0,34.0
2,Spain,190.0,35.0,10.0,2.0,0.0
3,United States,27.0,2.0,3.0,14.0,0.0
4,Canada,80.0,7.0,1.0,1.0,0.0


In [17]:
############################# mp_casedetection data handling ################################################
casedetection=pd.read_csv(mp_casedetection)
casedetection.rename(columns={'Hospitalised (Y/N/NA)':'isHospitalised','Isolated (Y/N/NA)':'isIsolated','Travel_history (Y/N/NA)':'hasTravelHistory'},inplace=True)
casedetection_transformed=casedetection

In [21]:
casedetection_transformed.head()

,Date_confirmation,Country,City,Age,Gender,Symptoms,isHospitalised,isIsolated,hasTravelHistory
0,2022-05-06,England,London,NaN,NaN,rash,Y,Y,Y
1,2022-05-12,England,London,NaN,NaN,rash,Y,Y,N
2,2022-05-13,England,London,NaN,NaN,vesicular rash,N,Y,N
3,2022-05-15,England,London,NaN,male,vesicular rash,Y,Y,N
4,2022-05-15,England,London,NaN,male,vesicular rash,Y,Y,N


In [29]:
connection_string = "postgres:1204@localhost:5432/monkeypox"
engine = create_engine(f'postgresql://{connection_string}')

In [30]:
engine.table_names()

C:\Users\vanem\AppData\Local\Temp\ipykernel_45340\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['individual', 'daily', 'worldwide', 'casedetection']

In [37]:
individual_transformed.to_sql(name='individual', con=engine, if_exists='append', index=True)

264

In [38]:
daily_transformed.to_sql(name='daily', con=engine, if_exists='append', index=True)

736

In [42]:
worldwide_transformed.to_sql(name='worldwide', con=engine, if_exists='append', index=True)

51

In [43]:
casedetection_transformed.to_sql(name='detection', con=engine, if_exists='append', index=True)

11